In [0]:
#import necessray packages
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [9]:
#import google drive to import data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#defining the function to draw most common words from training data
def make_Dictionary(root_dir):
  all_words = [] #set up an empty list
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] #extracting all files in folder
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words
  dictionary = Counter(all_words) #collecting all the texts in one file and count them
  list_to_remove = list(dictionary) #make the dictionary a list

  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item] #get rid of any items with non-alphabets
    elif len(item) == 1:
      del dictionary[item] #get rid of items that are single letters
  dictionary = dictionary.most_common(3000) #get words with highest frequencies
  return dictionary

In [0]:
train_1='/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
dictionary=make_Dictionary(train_1) #test dictionary with the training folder

In [0]:
def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)] 
  #extracting all files links from the folder in 'files'
  features_matrix = np.zeros((len(files),3000)) #create a matrix of 3000*len(files), all zeros
  train_labels = np.zeros(len(files)) #create a numpy array of zeros with len(files)
  count = 1;
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split() #split out words on the third line
          for word in words:
            wordID = 0 #ignore all words by assigning 0 first
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i #assign the wordID with its order in dictionary when it comes out 
                features_matrix[docID,wordID] = words.count(word) 
                #fill the count of that word in the matrix in [docID,wordID]
      train_labels[docID] = 0;
      filepathTokens = fil.split('/') #split file with '/'
      lastToken = filepathTokens[len(filepathTokens)-1] #pick the last word from file link
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1; #mark this file as spam in training_label
        count = count + 1 #count the spmsg files
      docID = docID + 1 #change docID bofore moving to the next file 
  return features_matrix, train_labels
  #feature_matrix rows stand for file number, columns stand for 3000 words
  #train_lavels are final results if a list of these files are spams, seperately

In [0]:
TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails'
dictionary = make_Dictionary(TRAIN_DIR) # using functions mentioned above.
features_matrix, labels = extract_features(TRAIN_DIR)
test_feature_matrix, test_labels = extract_features(TEST_DIR)

# from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
#train model
model.fit(features_matrix, labels)
#predict
predicted_labels = model.predict(test_feature_matrix)

In [19]:
# from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predicted_labels)

0.9615384615384616